In [36]:
import collections
import http.client
import pandas as pd
from operator import itemgetter
import json
from collections import namedtuple
pd.set_option("display.max_rows", 200)
pd.set_option('display.max_colwidth', -1)

In [37]:
Rounds = namedtuple('Rounds', 'player round score strokes thru')

In [38]:
conn = http.client.HTTPConnection("api.sportradar.us")
conn.request("GET", "/golf-t2/leaderboard/pga/2018/tournaments/b404a8d5-5e33-4417-ae20-5d4d147042ee/leaderboard.json?api_key=u2rg6jhkezj6dkutjcmfqc8e")
res = conn.getresponse()
data = res.read()
d = data.decode("utf-8")

In [39]:
missed_cut = ["Sandy Lyle", "Charl Schwartzel", "Jose Maria Olazabal", "Jason Dufner", "Patrick Cantlay", 
              "Shubhankar Sharma", "Thomas Pieters", "Ted Potter", "Brendan Steele", "Dylan Frittelli", 
              "Danny Willett", "Ross Fisher", "Wesley Bryan", "Pat Perez", "Patton Kizzire", "Trevor Immelman", 
              "Joaquin Niemann", "Yuta Ikeda", "Alexander Noren", "Kevin Chappell", "Doc Redman", "Gary Woodland", 
              "Yusaku Miyazato", "Austin Cook", "Billy Horschel", "Ian Woosnam", "Mike Weir", "Larry Mize", 
              "Angel Cabrera", "Yuxin Lin", "Mark O'Meara", "Matt Parziale", "Sergio Garcia", "Harry Ellis"]
missed_cut_players = [p.lower().replace(" ", "") for p in missed_cut]
# sorted(missed_cut_players)

In [40]:
cols = ["first_name", "last_name", "position", "score", "strokes", "full_name", "rounds"]
lb_df = pd.DataFrame(json.loads(d)['leaderboard'])
lb_df["full_name"] = lb_df["first_name"].str.lower().str.replace(" ", "") + lb_df["last_name"].str.lower().str.replace(" ", "")
lb_df[cols]
lb_df.index += 1
lb_df.to_csv("leaderboard.csv", columns=cols)
lb_df[[c for c in cols if c not in ("rounds",)]]

,first_name,last_name,position,score,strokes,full_name
1,Patrick,Reed,1,-14,202,patrickreed
2,Rory,McIlroy,2,-11,205,rorymcilroy
3,Rickie,Fowler,3,-9,207,rickiefowler
4,Jon,Rahm,4,-8,208,jonrahm
5,Henrik,Stenson,5,-7,209,henrikstenson
6,Tommy,Fleetwood,6,-6,210,tommyfleetwood
7,Marc,Leishman,6,-6,210,marcleishman
8,Bubba,Watson,6,-6,210,bubbawatson
9,Justin,Thomas,9,-5,211,justinthomas
10,Jordan,Spieth,9,-5,211,jordanspieth


In [41]:
sorted(list(lb_df[lb_df["status"] == 'CUT'].full_name)) == sorted(missed_cut_players)

True

In [54]:
players_and_rounds = [("{} {}".format(r["first_name"], r["last_name"]), 
                       r["rounds"][2], r["rounds"][3]) for r in json.loads(d)["leaderboard"] if len(r["rounds"]) > 2]
round3 = [Rounds(p[0], p[1]["sequence"], p[1]["score"], p[1]["strokes"], p[1]["thru"]) for p in players_and_rounds]
round4 = [Rounds(p[0], p[2]["sequence"], p[2]["score"], p[2]["strokes"], p[2]["thru"]) for p in players_and_rounds]
round4_df = pd.DataFrame(sorted(round4, key=lambda x: x.score))
round4_df.index += 1
round4_df.to_csv("round4_leaderboard.csv")
round4_df

,player,round,score,strokes,thru
1,Webb Simpson,4,-6,34,10
2,Phil Mickelson,4,-5,67,18
3,Brian Harman,4,-3,69,18
4,Ian Poulter,4,-3,69,18
5,Charley Hoffman,4,-2,14,4
6,Hideki Matsuyama,4,-2,25,7
7,Paul Casey,4,-2,25,7
8,Branden Grace,4,-2,50,13
9,Tony Finau,4,-1,15,4
10,Russell Henley,4,-1,26,7


In [45]:
def get_high_round(rounds):
    high_player = max(rounds, key=lambda x: x.strokes)
    return high_player
    
def get_score(player):
    player = player.lower().replace(" ", "")
    score = lb_df.loc[lb_df["full_name"] == player]["score"]
    strokes = lb_df.loc[lb_df["full_name"] == player]["strokes"]
    high_round3 = get_high_round(round3)
    high_round4 = get_high_round(round4)
    high_scores = high_round3.score + high_round4.score
    high_strokes = high_round3.strokes + high_round4.strokes
    if player in missed_cut_players:
        return "**MC** Score: {}, Strokes: {}".format(int(score) + high_scores, int(strokes) + high_strokes)
    return "Score: {}, Strokes: {}".format(int(score), int(strokes))
    
def get_strokes(player):
    player = player.lower().replace(" ", "")
    high_strokes = 0
    if player in missed_cut_players:
        high_strokes = get_high_round(round3).strokes + get_high_round(round4).strokes
        return int(lb_df.loc[lb_df["full_name"] == player]["strokes"]) + high_strokes
    else:
        return int(lb_df.loc[lb_df["full_name"] == player]["strokes"])

In [46]:
df = pd.read_csv("masters.csv")
df = df[["Pool Player", "Player1", "Player2", "Player3", "Player4", 
         "Player5", "Player6", "Player7", "Total", "Tie Breaker"]]

In [47]:
for index, row in df.iterrows():
    score_accum = 0
    for player in row.values[1:8]:
        score_accum += get_strokes(player)
    df.set_value(index, "Total", score_accum)

In [48]:
df = df.sort_values(by=['Total']).reset_index(drop=True)
for i in range(1,8):
    df["Score{}".format(i)] = df["Player{}".format(i)].apply(lambda x: get_score(x))

In [49]:
high_round3 = get_high_round(round3)
high_round4 = get_high_round(round4)
for r in (high_round3, high_round4,):  
    print("High round {} from {}, with score: {}, strokes: {}, thru: {}".format(r.round, r.player, r.score, r.strokes, r.thru))

High round 3 from Vijay Singh, with score: 7, strokes: 79, thru: 18
High round 4 from Chez Reavie, with score: 4, strokes: 76, thru: 18


In [50]:
df.index += 1
df = df[["Pool Player", 
         "Player1", "Score1",
         "Player2", "Score2",
         "Player3", "Score3",
         "Player4", "Score4",
         "Player5", "Score5",
         "Player6", "Score6",
         "Player7", "Score7",
         "Total", "Tie Breaker"]]
df

,Pool Player,Player1,Score1,Player2,Score2,Player3,Score3,Player4,Score4,Player5,Score5,Player6,Score6,Player7,Score7,Total,Tie Breaker
1,Paul Bower,Dustin Johnson,"Score: -4, Strokes: 212",Jason Day,"Score: -1, Strokes: 224",Bubba Watson,"Score: -6, Strokes: 210",Daniel Berger,"Score: 5, Strokes: 253",Adam Hadwin,"Score: 1, Strokes: 244",Tiger Woods,"Score: 4, Strokes: 267",Jimmy Walker,"Score: -1, Strokes: 224",1634.0,268.0
2,Sam Montgomery,Jordan Spieth,"Score: -5, Strokes: 211",Tommy Fleetwood,"Score: -6, Strokes: 210",Charley Hoffman,"Score: -3, Strokes: 229",Branden Grace,"Score: 2, Strokes: 270",Adam Hadwin,"Score: 1, Strokes: 244",Jimmy Walker,"Score: -1, Strokes: 224",Bryson DeChambeau,"Score: 5, Strokes: 273",1661.0,282.0
3,Jeremy Szyba,Rickie Fowler,"Score: -9, Strokes: 207",Paul Casey,"Score: 0, Strokes: 243",Bubba Watson,"Score: -6, Strokes: 210",Louis Oosthuizen,"Score: -3, Strokes: 217",Jason Dufner,"**MC** Score: 17, Strokes: 305",Jimmy Walker,"Score: -1, Strokes: 224",Zach Johnson,"Score: 2, Strokes: 258",1664.0,277.0
4,Joe Kearney,Justin Thomas,"Score: -5, Strokes: 211",Jason Day,"Score: -1, Strokes: 224",Patrick Reed,"Score: -14, Strokes: 202",Louis Oosthuizen,"Score: -3, Strokes: 217",Thomas Pieters,"**MC** Score: 18, Strokes: 306",Tiger Woods,"Score: 4, Strokes: 267",Russell Henley,"Score: -1, Strokes: 242",1669.0,279.0
5,Kelly McGrade,Jordan Spieth,"Score: -5, Strokes: 211",Jason Day,"Score: -1, Strokes: 224",Bubba Watson,"Score: -6, Strokes: 210",Louis Oosthuizen,"Score: -3, Strokes: 217",Thomas Pieters,"**MC** Score: 18, Strokes: 306",Tiger Woods,"Score: 4, Strokes: 267",Adam Scott,"Score: 1, Strokes: 249",1684.0,277.0
6,Tristen Woods,Jon Rahm,"Score: -8, Strokes: 208",Henrik Stenson,"Score: -7, Strokes: 209",Patrick Reed,"Score: -14, Strokes: 202",Brendan Steele,"**MC** Score: 18, Strokes: 306",Cameron Smith,"Score: -3, Strokes: 213",Billy Horschel,"**MC** Score: 22, Strokes: 310",Adam Scott,"Score: 1, Strokes: 249",1697.0,279.0
7,Jason Grill,Dustin Johnson,"Score: -4, Strokes: 212",Jason Day,"Score: -1, Strokes: 224",Bubba Watson,"Score: -6, Strokes: 210",Tony Finau,"Score: -2, Strokes: 230",Thomas Pieters,"**MC** Score: 18, Strokes: 306",Tiger Woods,"Score: 4, Strokes: 267",Adam Scott,"Score: 1, Strokes: 249",1698.0,280.0
8,Dave Peterson,Rory McIlroy,"Score: -11, Strokes: 205",Jason Day,"Score: -1, Strokes: 224",Patrick Reed,"Score: -14, Strokes: 202",Tony Finau,"Score: -2, Strokes: 230",Jason Dufner,"**MC** Score: 17, Strokes: 305",Tiger Woods,"Score: 4, Strokes: 267",Martin Kaymer,"Score: 4, Strokes: 281",1714.0,278.0
9,Pat McNamara,Justin Thomas,"Score: -5, Strokes: 211",Phil Mickelson,"Score: 2, Strokes: 290",Bubba Watson,"Score: -6, Strokes: 210",Daniel Berger,"Score: 5, Strokes: 253",Cameron Smith,"Score: -3, Strokes: 213",Tiger Woods,"Score: 4, Strokes: 267",Bryson DeChambeau,"Score: 5, Strokes: 273",1717.0,276.0
10,Zachary Sight,Justin Thomas,"Score: -5, Strokes: 211",Phil Mickelson,"Score: 2, Strokes: 290",Patrick Reed,"Score: -14, Strokes: 202",Daniel Berger,"Score: 5, Strokes: 253",Siwoo Kim,"Score: 0, Strokes: 232",Tiger Woods,"Score: 4, Strokes: 267",Fred Couples,"Score: 3, Strokes: 263",1718.0,278.0


In [51]:
df.to_csv("masters_results.csv")

# Tabula Fun

In [52]:
from tabula import read_pdf

In [53]:
df = read_pdf("/Users/jeremyszyba/Desktop/2018 Masters.pdf")
df = df.drop(df.index[10])
df

,Group 1,Group 2,Group 3,Group 4,Group 5,Field,Field.1,Field.2,Field.3
0,Dustin Johnson,Tommy Fleetwood,Bubba Watson,Branden Grace,Li Haotong,Ryan Armour,Mike Weir,Patton Kizzire,Lin Yuxin
1,Justin Thomas,Jason Day,Rafael Cabrera Bello,Louis Oosthuizen,Adam Hadwin,Daniel Berger,Bernd Wiesberger,Bernhard Langer,Doug Ghim
2,Jon Rahm,Paul Casey,Brian Harman,Patrick Cantlay,Kyle Stanley,Wesley Bryan,Danny Willett,Sandy Lyle,Russell Henley
3,Jordan Spieth,Alex Noren,Patrick Reed,Tony Finau,Cameron Smith,Angel Cabrera,Tiger Woods,Matt Parziale,Billy Horschel
4,Justin Rose,Henrik Stenson,Kevin Kisner,Ross Fisher,Thomas Pieters,Austin Cook,Ian Woosnam,Doc Redman,Larry Mize
5,Hideki Matsuyama,Marc Leishman,Xander Schauffele,Matthew Fitzpatrick,Satoshi Kodaira,Fred Couples,Yuta Ikdea,Ted Potter Jr.,Vijay Singh
6,Rory McIlroy,Tyrrell Hatton,Francesco Molinari,Daniel Berger,Dylan Frittelli,Bryson DeChambeau,Trevor Immelman,Charl Schwartzel,Jose Maria Olazabal
7,Rickie Fowler,Phil Mickelson,Charley Hoffman,Kevin Chappell,Chez Reavie,Harry Ellis,Zach Johnson,Adam Scott,Ryan Moore
8,Sergio Garcia,Pat Perez,Kiradech Aphibarnrat,Brendan Steele,Jason Dufner,Jhonattan Vegas,Martin Kaymer,Shubanhkar Sharma,Joaquin Niemann
9,Brooks Koepka,Matt Kuchar,Gary Woodland,Webb Simpson,Siwoo Kim,Jimmy Walker,Yusaku Miyazato,Webb Simpson,Mark O'Meara
